# Yolo bottom view training

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Rotation argumentation of bottom view

All argumentation

In [ ]:
!pip install albumentations opencv-python scikit-learn tqdm pyyaml

import os
import cv2
import glob
import shutil
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import albumentations as A
from collections import Counter
from pathlib import Path
import yaml

# ===============================================================
# === CONFIGURATION =============================================
# ===============================================================
DATASET_DIR = "/content/drive/MyDrive/For thesis/Defect Detection/all argumentation/Bottom view/Bottom view.v2i.yolov8 (1)/train"
OUTPUT_DIR = "/content/drive/MyDrive/For thesis/Defect Detection/all argumentation/Bottom view"
YAML_TEMPLATE = "/content/drive/MyDrive/For thesis/Defect Detection/all argumentation/Bottom view/Bottom view.v2i.yolov8 (1)/data.yaml"

SPLIT_RATIOS = (0.7, 0.2, 0.1)  # train, val, test
ROTATION_ANGLES = list(range(-15, 20, 5))  # [-15, -10, -5, 0, 5, 10, 15]

# ===============================================================
# === STEP 1: SPLIT DATASET =====================================
# ===============================================================
IMG_DIR = os.path.join(DATASET_DIR, "images")
LAB_DIR = os.path.join(DATASET_DIR, "labels")

image_paths = []
for ext in ["*.jpg", "*.png", "*.jpeg"]:
    image_paths.extend(glob.glob(os.path.join(IMG_DIR, ext)))

image_paths = sorted(image_paths)
print(f"📂 Found {len(image_paths)} total images")

train_paths, temp_paths = train_test_split(image_paths, test_size=(1 - SPLIT_RATIOS[0]), random_state=42)
val_paths, test_paths = train_test_split(temp_paths, test_size=SPLIT_RATIOS[2] / (SPLIT_RATIOS[1] + SPLIT_RATIOS[2]), random_state=42)
splits = {"train": train_paths, "val": val_paths, "test": test_paths}

for split in splits:
    os.makedirs(os.path.join(OUTPUT_DIR, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(OUTPUT_DIR, split, "labels"), exist_ok=True)

for split, paths in splits.items():
    print(f"Copying {split} set ({len(paths)} images)...")
    for img_path in tqdm(paths):
        base = os.path.splitext(os.path.basename(img_path))[0]
        label_path = os.path.join(LAB_DIR, base + ".txt")
        shutil.copy(img_path, os.path.join(OUTPUT_DIR, split, "images", os.path.basename(img_path)))
        if os.path.exists(label_path):
            shutil.copy(label_path, os.path.join(OUTPUT_DIR, split, "labels", base + ".txt"))
        else:
            open(os.path.join(OUTPUT_DIR, split, "labels", base + ".txt"), "w").close()

print("✅ Dataset split complete.")

# ===============================================================
# === STEP 2: DEFINE AUGMENTATIONS ===============================
# ===============================================================

# General augmentations (original)
general_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.3),
    A.RandomBrightnessContrast(p=0.5),
    A.HueSaturationValue(p=0.3),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
    A.MotionBlur(blur_limit=3, p=0.3),
    A.RandomGamma(p=0.3),
], bbox_params=A.BboxParams(
    format="yolo",
    label_fields=["class_labels"],
    min_visibility=0.1
))

# ===============================================================
# === STEP 3: APPLY ROTATION + GENERAL AUGMENTATIONS ============
# ===============================================================
TRAIN_IMG_DIR = os.path.join(OUTPUT_DIR, "train", "images")
TRAIN_LAB_DIR = os.path.join(OUTPUT_DIR, "train", "labels")
AUG_IMG_DIR = os.path.join(OUTPUT_DIR, "train_aug", "images")
AUG_LAB_DIR = os.path.join(OUTPUT_DIR, "train_aug", "labels")

os.makedirs(AUG_IMG_DIR, exist_ok=True)
os.makedirs(AUG_LAB_DIR, exist_ok=True)

def load_yolo_labels(label_path):
    boxes, classes = [], []
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for line in f.readlines():
                parts = line.strip().split()
                if len(parts) == 5:
                    cls, x, y, w, h = parts
                    boxes.append([float(x), float(y), float(w), float(h)])
                    classes.append(int(float(cls)))
    return boxes, classes

train_images = []
for ext in ["*.jpg", "*.png", "*.jpeg"]:
    train_images.extend(glob.glob(os.path.join(TRAIN_IMG_DIR, ext)))

print(f"\n🌀 Applying rotation + general augmentations to {len(train_images)} training images...")

orig_class_counts = Counter()
aug_class_counts = Counter()

for img_path in tqdm(train_images):
    img = cv2.imread(img_path)
    if img is None:
        continue
    base = os.path.splitext(os.path.basename(img_path))[0]
    label_path = os.path.join(TRAIN_LAB_DIR, base + ".txt")
    boxes, classes = load_yolo_labels(label_path)

    # BACKGROUND IMAGES
    if len(boxes) == 0:
        for angle in ROTATION_ANGLES:
            rot = A.Rotate(limit=(angle, angle), border_mode=cv2.BORDER_CONSTANT, value=0, p=1)
            rot_img = rot(image=img)["image"]
            cv2.imwrite(os.path.join(AUG_IMG_DIR, f"{base}_rot{angle}.jpg"), rot_img)
            open(os.path.join(AUG_LAB_DIR, f"{base}_rot{angle}.txt"), "w").close()

        for i in range(2):
            aug_img = general_transform(image=img, bboxes=[], class_labels=[])["image"]
            cv2.imwrite(os.path.join(AUG_IMG_DIR, f"{base}_aug{i}.jpg"), aug_img)
            open(os.path.join(AUG_LAB_DIR, f"{base}_aug{i}.txt"), "w").close()
        continue

    # DEFECT IMAGES
    orig_class_counts.update(classes)

    for angle in ROTATION_ANGLES:
        rot_transform = A.Compose([
            A.Rotate(limit=(angle, angle), border_mode=cv2.BORDER_CONSTANT, value=0, p=1)
        ], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"], min_visibility=0.1))

        rotated = rot_transform(image=img, bboxes=boxes, class_labels=classes)
        rot_img, rot_boxes, rot_classes = rotated["image"], rotated["bboxes"], rotated["class_labels"]

        cv2.imwrite(os.path.join(AUG_IMG_DIR, f"{base}_rot{angle}.jpg"), rot_img)
        with open(os.path.join(AUG_LAB_DIR, f"{base}_rot{angle}.txt"), "w") as f:
            for cls, bbox in zip(rot_classes, rot_boxes):
                x, y, w, h = [min(max(v, 0), 1) for v in bbox]
                f.write(f"{cls} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")
        aug_class_counts.update(rot_classes)

    # Add 2 general augmentations
    for i in range(2):
        augmented = general_transform(image=img, bboxes=boxes, class_labels=classes)
        aug_img, aug_boxes, aug_classes = augmented["image"], augmented["bboxes"], augmented["class_labels"]
        cv2.imwrite(os.path.join(AUG_IMG_DIR, f"{base}_aug{i}.jpg"), aug_img)
        with open(os.path.join(AUG_LAB_DIR, f"{base}_aug{i}.txt"), "w") as f:
            for cls, bbox in zip(aug_classes, aug_boxes):
                x, y, w, h = [min(max(v, 0), 1) for v in bbox]
                f.write(f"{cls} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")
        aug_class_counts.update(aug_classes)

print("✅ Rotation + augmentation complete.")

# ===============================================================
# === STEP 4: MERGE train + train_aug ===========================
# ===============================================================
FINAL_IMG_DIR = os.path.join(OUTPUT_DIR, "train_final", "images")
FINAL_LAB_DIR = os.path.join(OUTPUT_DIR, "train_final", "labels")
os.makedirs(FINAL_IMG_DIR, exist_ok=True)
os.makedirs(FINAL_LAB_DIR, exist_ok=True)

def merge_folders(src_img, src_lab, dst_img, dst_lab):
    for src_dir, dst_dir in [(src_img, dst_img), (src_lab, dst_lab)]:
        files = glob.glob(os.path.join(src_dir, "*"))
        for f in tqdm(files, desc=f"Merging {os.path.basename(src_dir)}"):
            shutil.copy(f, os.path.join(dst_dir, os.path.basename(f)))

merge_folders(TRAIN_IMG_DIR, TRAIN_LAB_DIR, FINAL_IMG_DIR, FINAL_LAB_DIR)
merge_folders(AUG_IMG_DIR, AUG_LAB_DIR, FINAL_IMG_DIR, FINAL_LAB_DIR)

print("✅ Merged into train_final")

# ===============================================================
# === STEP 5: AUTO-GENERATE data.yaml ===========================
# ===============================================================
with open(YAML_TEMPLATE, "r") as f:
    data = yaml.safe_load(f)

data["train"] = str(Path(OUTPUT_DIR) / "train_final/images")
data["val"]   = str(Path(OUTPUT_DIR) / "val/images")
data["test"]  = str(Path(OUTPUT_DIR) / "test/images")

yaml_out = Path(OUTPUT_DIR) / "data.yaml"
with open(yaml_out, "w") as f:
    yaml.safe_dump(data, f)

print(f"\n✅ New YOLO data.yaml saved at:\n{yaml_out}")
print(f"Train path: {data['train']}")
print(f"Val path:   {data['val']}")
print(f"Test path:  {data['test']}")
print("\n📊 Class Distribution:")
print("Original:", dict(orig_class_counts))
print("After Augmentation:", dict(orig_class_counts + aug_class_counts))


📂 Found 44 total images
Copying train set (30 images)...


100%|██████████| 30/30 [00:25<00:00,  1.19it/s]


Copying val set (9 images)...


100%|██████████| 9/9 [00:07<00:00,  1.23it/s]


Copying test set (5 images)...


...

✅ Dataset split complete.

🌀 Applying rotation + general augmentations to 30 training images...


  0%|          | 0/30 [00:00<?, ?it/s]/tmp/ipython-input-1435631685.py:136: UserWarning: Argument(s) 'value' are not valid for transform Rotate
  A.Rotate(limit=(angle, angle), border_mode=cv2.BORDER_CONSTANT, value=0, p=1)
100%|██████████| 30/30 [00:07<00:00,  4.20it/s]


✅ Rotation + augmentation complete.


Merging labels: 100%|██████████| 270/270 [00:02<00:00, 108.12it/s]


✅ Merged into train_final

✅ New YOLO data.yaml saved at:
/content/drive/MyDrive/For thesis/Defect Detection/all argumentation/Bottom view/data.yaml
Train path: /content/drive/MyDrive/For thesis/Defect Detection/all argumentation/Bottom view/train_final/images
Val path:   /content/drive/MyDrive/For thesis/Defect Detection/all argumentation/Bottom view/val/images
Test path:  /content/drive/MyDrive/For thesis/Defect Detection/all argumentation/Bottom view/test/images

📊 Class Distribution:
Original: {0: 66}
After Augmentation: {0: 660}


In [ ]:
# Install YOLO if not already
!pip install ultralytics

from ultralytics import YOLO
import os

# ---------------------------
# Config
# ---------------------------
DATA_YAML = "/content/drive/MyDrive/For thesis/Defect Detection/all argumentation/Bottom view/data.yaml"
# 👆 this is the NEW yaml created from your Roboflow template with updated train/val/test paths

MODEL = "yolov8s.pt"          # pretrained backbone
EPOCHS = 100                  # total epochs
BATCH = 8                     # adjust if you hit OOM
IMGSZ = 1920                  # high res for small defects
PATIENCE = 30                 # early stopping patience

SAVE_DIR = "/content/drive/MyDrive/For thesis/Defect Detection/all argumentation/Bottom view"

# ---------------------------
# Train (with resume support)
# ---------------------------

# Path to last run checkpoint (Ultralytics auto-saves best + last.pt)
last_ckpt = os.path.join(SAVE_DIR, "results/weights/last.pt")

if os.path.exists(last_ckpt):
    print("🔄 Resuming training from last checkpoint...")
    model = YOLO(last_ckpt)
    model.train(
        resume=True
    )
else:
    print("🚀 Starting fresh training with YOLOv8s...")
    model = YOLO(MODEL)  # load pretrained YOLOv8s backbone
    model.train(
        data=DATA_YAML,
        epochs=EPOCHS,
        batch=BATCH,
        imgsz=IMGSZ,
        patience=PATIENCE,
        project=SAVE_DIR,
        name="train",
        exist_ok=True,
        amp=True  # mixed precision
    )


🚀 Starting fresh training with YOLOv8s...
Ultralytics 8.3.209 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/For thesis/Defect Detection/all argumentation/Bottom view/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1920, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=N